In [ ]:
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
# API_KEY = os.getenv("CONGRESS_API_KEY")
API_KEY = "cW09gbNMaffaTf3nQczNUKl7XZt5lgHYL40uDDMn" # cadens

BASE_URL = "https://api.congress.gov/v3"
endpoint = "/bill"
congresses = [i for i in range(106, 120)] # year 2000-2025
congresses.reverse()
bill_types = ["hr", "s", "hjres", "sjres"] # house of reps, senate, house joint res, senate joint res
params = {"api_key" : API_KEY,
          "limit" : 1} # 250 is max

COMPLETED = []

congresses

In [ ]:
public_law_count = 0
unique_bills_with_text = 0
total_texts = 0
bill_count = 0
bills_with_recorded_votes = 0
bills_with_text_and_recorded_votes = 0
bills_with_text_and_recorded_votes_that_became_public_law = 0

for c in congresses:
    if c != 118:
        continue
    if c in COMPLETED:
        print(f"{c} already done")
        continue
    for b_t in bill_types:
        if b_t != "hr":
            continue
        file_name = f"compiled_json/{c}_{b_t}_compiled.json"
        if os.path.exists(file_name):
            print(f"{file_name} exists")
            with open(file_name, "r") as file:
                data = json.load(file)
                for bill in data:
                    if "actions" in data[bill]:
                        actions = data[bill]["actions"]
                        if "error" not in actions: # actions have been taken
                            if "actions" in actions:
                                for i in range(len(actions["actions"])):
                                    a = actions["actions"][i]
                                    if "recordedVotes" in a:
                                        print(bill)
                                        bills_with_recorded_votes += 1
                                        break
                    bill_count += 1
                    if "error" in data[bill]:
                        continue
                    if "error" in data[bill]["text"]:
                        continue
                    if "textVersions" not in data[bill]["text"]:
                        # print(bill, "no text version")
                        continue
                    one_text = True
                    for v in data[bill]["text"]["textVersions"]:
                        print(v)
                        if one_text:
                            unique_bills_with_text += 1
                            if "actions" in data[bill]:
                                actions = data[bill]["actions"]
                                if "error" not in actions: # actions have been taken
                                    if "actions" in actions:
                                        for i in range(len(actions["actions"])):
                                            a = actions["actions"][i]
                                            if "recordedVotes" in a:
                                                bills_with_text_and_recorded_votes += 1
                                                break
                            one_text = False
                        if "type" in v and v["type"] == "Public Law":
                            if "actions" in data[bill]:
                                actions = data[bill]["actions"]
                                if "error" not in actions: # actions have been taken
                                    if "actions" in actions:
                                        for i in range(len(actions["actions"])):
                                            a = actions["actions"][i]
                                            if "recordedVotes" in a:
                                                bills_with_text_and_recorded_votes_that_became_public_law += 1
                                                break
                            public_law_count += 1
                        if len(v["formats"]) > 0:
                            # print(bill, v["formats"][0]["url"])
                            total_texts += 1
                        # else:
                        #     print(bill, "no associated text")                

In [ ]:
print("bills with text that became public law:", public_law_count)
print("number of bills that contain at least one text:", unique_bills_with_text)
print("total texts available across all bills:", total_texts)
print("total bills:", bill_count)
print("bills with recorded votes", bills_with_recorded_votes)
print("bills with text and recorded votes", bills_with_text_and_recorded_votes)
print("bills with text and recorded votes that became public law", bills_with_text_and_recorded_votes_that_became_public_law)